In [1]:
from __future__ import print_function, division

In [2]:
import ads
import sys
import numpy
import unicodedata
from StringIO import StringIO
from time import localtime

In [3]:
ads.config.token = 'zmmgkG86fM0hBeBjoOJicrr5GegJWe69vJ5ULDNd'

origstafflist=["Hood, A. W.", "Archontis, V.","De Moortel, I.","Mackay, D. H.","Neukirch, T.",
           "Parnell, C. E.","Wright, A. N.","Conlon, T. M.","Antolin, P.", "Pagano, P.",
           "Threlfall, J.","Wilson, F.", "Haynes, A. L.", "Brooks, S.", "Fruchtl, H.",
           "MacFarlane, F.", "Stevenson, J. E. H.", "Elsden, T.", "Hodgson, J. D. B.","Lee, E. J.", 
           "O'Hara, J.", "Allanson, O.", "Sturrock, Z.", "Borissov, A.","Johnston, C. D. J.",
           "Williams, B. M.","Goldstraw, E.","Howson, T.", "Priest, E. R.",
           "Cargill, P. J.","Roberts, B."]

In [4]:
def load(fname):
    ''' load the file using std open'''
    f = open(fname,'r')

    data = []
    for line in f.readlines():
        data.append(line.replace('\n','').split(','))

    f.close()

    return data

In [5]:
def compareauthor(name):
    '''compare author with SMTG grp members and return link if matched'''    
    assert type(name) == unicode, 'name must be unicode string'
    
    staff = load('/Users/james/Public/smtg/staff_current.txt')
    found=0
    for people in staff:
        nom=','.join(people[2:4])[2:-1]
        if  nom == name.encode("utf-8"):
            #print('FOUND'),
            found=1
            jsthing=''.join(people[7:])
            #print(jsthing[1:]),
            linktoreturn = '<a href={0}>{1}</a>'.format(jsthing[1:],name)
            type(linktoreturn)
            #print(linktoreturn),
            #linktoreturn="<a href=\""+''.join(people[7:])+">"
            #linktoreturn=linktoreturn+name+"</a>"

    if found == 0:
        linktoreturn=name
    
    linktoreturn=u'{0}; '.format(linktoreturn)
            
    return linktoreturn

def old_print_table(query, filename):

    target = open(filename, 'w')
    target.truncate()
    
    pretty_author_name = lambda author: author.split(",")[0] + author.split(",")[1].strip()[1] + "."
    my_string = '' 
    target.write('<table>')
    target.write("\n")
    #print('<table>')
    for paper in query:
        authors=paper.author
        title=paper.title
        target.write('<tr><td>')
        target.write("\n")
        #print ('<tr><td>')
        for author in authors:
            temp=compareauthor(name=author.encode('utf-8'))
            target.write(temp)
            target.write("\n")
            #print(temp, end=""),
        
        target.write('</td><td>')
        #target.write(my_string.join(title))
        target.write(' '.join(title).encode('utf-8'))
        target.write('</td></tr>')
        target.write("\n")
        #print('</td><td>')
        #print(my_string.join(title), end=""),
        #print('</td></tr>')
    
    target.write('</table>')
    target.close()
    #print('</table>')

now = localtime()
year, month = (now.tm_year, now.tm_mon - 1)
if 1 > month: year, month = (year - 1, month)
print('Querying Year/Month {0} (in month {1})'.format(year, month)),
print(),

    # Retrieve all the articles published last month.
articles = ads.SearchQuery( q='aff:(\"University of St Andrews\") \
    property:refereed pubdate:{0} database:("astronomy" OR "physics")'.format(year),
    fl=['id', 'first_author', 'year', 'bibcode', 'identifier', 'author','title'],
    sort='read_count desc')


print_table(articles,'test.html')

In [6]:
def write_table(tlist,alist, filename):
    '''output simple html table containing list of recent SMTG pubs and authors'''
    assert type(tlist) == list, 'titlelist must be list of titles'
    assert type(alist) == list, 'authorlist must be list of authors'
    assert len(tlist) == len(alist), 'author and title list must be same length'
    assert type(filename) == str, 'name must be string'
    
    target = open(filename, 'w')
    target.truncate()
    
    #pretty_author_name = lambda author: author.split(",")[0] + author.split(",")[1].strip()[1] + "."
    my_string = '' 
    target.write('<HTML><head></head><body>')
    target.write('<table>')
    target.write("\n")
    for title, authors in zip(tlist, alist):    
    #for paper in titlelist:
        target.write('<tr><td>')
        target.write("\n")
        for author in authors:
            temp=compareauthor(name=author)
            target.write(temp.encode('ascii', 'xmlcharrefreplace'))
            target.write("\n")
    
        target.write('</td><td>')
        #target.write(my_string.join(title))
        target.write(' '.join(title).encode('ascii', 'xmlcharrefreplace'))
        target.write('</td></tr>')
        target.write("\n")
    
    target.write('</table></body></HTML>')
    target.close()

In [7]:
#what if we loop over authors and store info?
now = localtime()
year, month = (now.tm_year, now.tm_mon - 1)
if 1 > month: year, month = (year - 1, month)
print('Querying Year/Month {0} (in month {1})'.format(year, month)),
print(),

staff = load('/Users/james/Public/smtg/staff_current.txt')

authlist=[]
titlelist=[]

for person in staff:
    eachname=','.join(person[2:4])[2:-1]

    # Retrieve all the articles published last month.
    query = ads.SearchQuery( q='author:\"{0}\" pubdate:{1} \
    aff:(\"University of St Andrews\") property:refereed  \
    database:("astronomy" OR "physics")'.format(eachname, year),
    #fl=['id', 'first_author', 'year', 'bibcode', 'identifier', 'author','title'],
    fl=['author','title'])

    query.execute()
    num = int(query.response.numFound)
    print("{person} had {num} publications in {year}".format(person=eachname, num=num, year=year))
  
    #staff_data["articles"].append([year, num])
    #staff_data["total"] += num
    #type(articles)
    if num > 0:
        for n in query:
            #print(n.title)
            if n.title not in titlelist:
                titlelist.append(n.title)
                authlist.append(n.author)
            #else:
                #print('matched publication, ignoring..')

    #staff_data["titles"].append([])
write_table(tlist=titlelist,alist=authlist,filename='test.html')    
#print_table(articles,'test.html')

Querying Year/Month 2016 (in month 4)

Hood, A. W. had 3 publications in 2016
Archontis, V. had 0 publications in 2016
De Moortel, I. had 2 publications in 2016
Mackay, D. H. had 2 publications in 2016
Neukirch, T. had 4 publications in 2016
Parnell, C. E. had 3 publications in 2016
Wright, A. N. had 2 publications in 2016
Conlon, T. M. had 0 publications in 2016
Antolin, P. had 1 publications in 2016
Pagano, P. had 0 publications in 2016
Threlfall, J. had 2 publications in 2016
Wilson, F. had 1 publications in 2016
Haynes, A. L. had 0 publications in 2016
Brooks, S. had 0 publications in 2016
Fruchtl, H. had 0 publications in 2016
MacFarlane, F. had 0 publications in 2016
Stevenson, J. had 1 publications in 2016
Elsden, T. had 0 publications in 2016
Hodgson, J. had 0 publications in 2016
Lee, E. J. had 0 publications in 2016
O'Hara, J. had 0 publications in 2016
Allanson, O. had 0 publications in 2016
Sturrock, Z. had 0 publications in 2016
Borissov, A. had 0 publications in 2016
John

/Users/james/anaconda/lib/python2.7/site-packages/ads/utils.py:23: UserWarning: You are lazy loading attributes via 'title', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,
/Users/james/anaconda/lib/python2.7/site-packages/ads/utils.py:23: UserWarning: You are lazy loading attributes via 'author', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,


In [8]:
print(authlist)

[[u'De Moortel, I.', u'Pascoe, D. J.', u'Wright, A. N.', u'Hood, A. W.'], [u'Hood, A. W.', u'Cargill, P. J.', u'Browning, P. K.', u'Tam, K. V.'], [u'Bareford, M. R.', u'Gordovskyy, M.', u'Browning, P. K.', u'Hood, A. W.'], [u'Cargill, P. J.', u'De Moortel, I.', u'Kiddie, G.'], [u'Gibb, G. P. S.', u'Mackay, D. H.', u'Jardine, M. M.', u'Yeates, A. R.'], [u'Jess, David B.', u'Reznikova, Veronika E.', u'Ryans, Robert S. I.', u'Christian, Damian J.', u'Keys, Peter H.', u'Mathioudakis, Mihalis', u'Mackay, Duncan H.', u'Krishna Prasad, S.', u'Banerjee, Dipankar', u'Grant, Samuel D. T.', u'Yau, Sean', u'Diamond, Conor'], [u'Wilson, F.', u'Neukirch, T.', u'Hesse, M.', u'Harrison, M. G.', u'Stark, C. R.'], [u'Threlfall, J.', u'Stevenson, J. E. H.', u'Parnell, C. E.', u'Neukirch, T.'], [u'Threlfall, J.', u'Bourdin, Ph. -A.', u'Neukirch, T.', u'Parnell, C. E.'], [u'MacTaggart, D.', u'Gregory, S. G.', u'Neukirch, T.', u'Donati, J. -F.'], [u'Edwards, S. J.', u'Parnell, C. E.', u'Harra, L. K.', u'Cul